# Simple Capacity 
Like the [Capacity Notebook](capacity.ipynb) but hardcoded models

In [2]:
from azure.identity import DefaultAzureCredential
import requests,json,os
import pandas as pd
import time

print(time.strftime("%Y-%m-%d %H:%M:%S"))

credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default").token
headers= {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}

2024-09-12 08:21:41


In [3]:
subs = requests.get('https://management.azure.com/subscriptions?api-version=2022-12-01',headers=headers).json()
if 'value' in subs:
    subId=subs['value'][0]['subscriptionId']
    print(f"Using subscription {subId}")
else:
    print("No subscriptions found")

Using subscription cd60139f-77b6-4946-8ff6-7c2135d571fa


In [4]:
def GetModelCapacities(subId,modelVersion,modelName,modelFormat):

    uri=f"https://management.azure.com/subscriptions/{subId}/providers/Microsoft.CognitiveServices/modelCapacities?api-version=2024-04-01-preview&modelFormat={modelFormat}&modelName={modelName}&modelVersion={modelVersion}"
    items=[]

    while uri:
        request=requests.get(uri, headers=headers,timeout=15)
        
        #loop through the value attribute of the response and generate embeddings for each content
        if 200 == request.status_code:
            response=request.json()
            for val in response['value']:
                item={}
                item['id']=val['id']
                item['name']=val['name']
                item['location']=val['location']
                item['modelName']=modelName
                item['modelVersion']=modelVersion
                item['modelFormat']=modelFormat
                item['availableCapacity']=val['properties']['availableCapacity']
                items.append(item)

            #check if the odata.nextLink attribute is present in the response
            if 'nextLink' in response:
                uri=response['nextLink']
            else:
                uri=None
        else:
            uri=None
    return items


In [5]:
models=[]

models.append({'model':'gpt-35-turbo','version':'0125','kind':'OpenAI','increment':50})
models.append({'model':'gpt-35-turbo','version':'1106','kind':'OpenAI','increment':50})
models.append({'model':'gpt-4','version':'0125-Preview','kind':'OpenAI','increment':100})
models.append({'model':'gpt-4','version':'0613','kind':'OpenAI','increment':100})
models.append({'model':'gpt-4-32k','version':'0613','kind':'OpenAI','increment':200})
models.append({'model':'gpt-4','version':'1106-Preview','kind':'OpenAI','increment':100})
models.append({'model':'gpt-4','version':'turbo-2024-04-09','kind':'OpenAI','increment':100})
models.append({'model':'gpt-4o','version':'2024-05-13','kind':'OpenAI','increment':50})
models.append({'model':'gpt-4o-mini','version':'2024-07-18','kind':'OpenAI','increment':25})
models.append({'model':'gpt-4o','version':'2024-08-06','kind':'OpenAI','increment':50})

In [ ]:
capacities=[]
# for model,version,kind in models:
for model,version,kind in list(set([(d['model'],d['version'],d['kind']) for d in models])):
    print(model,version,kind)
    try:
        items = GetModelCapacities(subId,version,model,kind)
        capacities.extend(items)
    except Exception as ex:
        print(ex)

# print(capacities)
# file_path = f'Files/modelCapacities.json'
# os.makedirs(os.path.dirname(file_path), exist_ok=True)
# with open(file_path, 'w') as f:
#     json.dump(capacities, f)

In [7]:
df = pd.DataFrame(capacities)
# Filter out the ones with no capacity
df = df[df['availableCapacity'] > 0]
df = df[df['name'] == 'ProvisionedManaged']
# Filter out the ones that are not gpt-4o
df = df[df['modelName'].str.startswith('gpt-4o')]
# List by location and model and sort by that too
df = df.sort_values(by=['location','modelName','modelVersion','modelFormat'])
# display without wrapping
pd.set_option('display.expand_frame_repr', False)
# display the location,modelName,modelVersion,modelFormat,availableCapacity
df = df[['location','modelName','modelVersion','modelFormat','availableCapacity']]
print(df)


               location    modelName modelVersion modelFormat  availableCapacity
112         brazilsouth       gpt-4o   2024-05-13      OpenAI                100
57          brazilsouth  gpt-4o-mini   2024-07-18      OpenAI                100
153          canadaeast       gpt-4o   2024-05-13      OpenAI                100
70           canadaeast  gpt-4o-mini   2024-07-18      OpenAI                 13
127             eastus2       gpt-4o   2024-05-13      OpenAI                100
63              eastus2  gpt-4o-mini   2024-07-18      OpenAI                100
147  germanywestcentral       gpt-4o   2024-05-13      OpenAI                100
129           japaneast       gpt-4o   2024-05-13      OpenAI                100
133        koreacentral       gpt-4o   2024-05-13      OpenAI                100
138      northcentralus       gpt-4o   2024-05-13      OpenAI                100
64       northcentralus  gpt-4o-mini   2024-07-18      OpenAI                100
120      southcentralus     